**Table of contents**<a id='toc0_'></a>    
- [Import statements](#toc1_1_)    
- [Loading the datasets](#toc1_2_)    
- [Non-parametric tests](#toc2_)    
  - [Wilcoxon signed-rank test](#toc2_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=5
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

### <a id='toc1_1_'></a>[Import statements](#toc0_)

In [4]:
import warnings

warnings.filterwarnings("ignore")

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### <a id='toc1_2_'></a>[Loading the datasets](#toc0_)

- The *"dem_votes_potus_12_16"* dataset contains the percentage of votes for the Democratic candidate in the 2012 and 2016 presidential elections for each county in the United States. The "dem_percent_2012" column contains the percentage of votes for the Democratic candidate in the 2012 election, and the "dem_percent_2016" column contains the percentage of votes for the Democratic candidate in the 2016 election.

In [6]:
sample_dem_data = pd.read_feather("./datasets/dem_votes_potus_12_16.feather")

In [7]:
sample_dem_data.head()

,state,county,dem_percent_12,dem_percent_16
0,Alabama,Bullock,76.305900,74.946921
1,Alabama,Chilton,19.453671,15.847352
2,Alabama,Clay,26.673672,18.674517
3,Alabama,Cullman,14.661752,10.028252
4,Alabama,Escambia,36.915731,31.020546


In [8]:
sample_dem_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   state           500 non-null    object 
 1   county          500 non-null    object 
 2   dem_percent_12  500 non-null    float64
 3   dem_percent_16  500 non-null    float64
dtypes: float64(2), object(2)
memory usage: 15.8+ KB


## <a id='toc2_'></a>[Non-parametric tests](#toc0_)

The tests that we've seen so far are known as parametric tests. Tests like the z-test, t-test, and ANOVA are all based on the assumption that the population is normally distributed. Parametric tests also require sample sizes that are "big enough" that the Central Limit Theorem applies. 

To check if the assumptions for hypothesis testing holds we can perform a sanity check. We calculate a bootstrap distribution and visualize it with a histogram. If we don't see a bell-shaped normal curve, then one of the assumptions hasn't been met. In that case, we should revisit the data collection process, and see if any of the three assumptions of randomness, independence, and sample size do not hold.

In situations where we aren't sure about these assumptions, or we are certain that the assumptions aren't met, we can use non-parametric tests. They do not make the normal distribution assumptions and does not require the sample size conditions. 

There are various non-parametric tests such as the the Wilcoxon signed-rank test, Mann-Whitney U test, Kruskal-Wallis test, and the Spearman's rank correlation test etc. which act as alternatives to their parametric counterparts.

### <a id='toc2_1_'></a>[Wilcoxon signed-rank test](#toc0_)

The Wilcoxon signed-rank test works well when the assumptions of a paired t-test aren't met.

The Wilcoxon signed-rank test is a non-parametric statistical test used to compare the distribution of a continuous variable before and after an intervention, or between two paired groups. It is a non-parametric test, meaning that it does not make any assumptions about the distribution of the data.

The Wilcoxon signed-rank test is similar to the paired t-test, but it is more robust to violations of the assumptions of normality and homogeneity of variance. It is also more powerful than the paired t-test when the sample size is small.

To perform the Wilcoxon signed-rank test, the following steps are taken:

- Calculate the difference between the paired observations.
- Rank the absolute values of the differences, from smallest to largest.
- Assign a sign to each rank, depending on whether the difference was positive or negative.
- Calculate the Wilcoxon signed-rank statistic (W) by summing the signed ranks i.e, $ W = min(\text{sum of positive ranks}, \text{sum of negative ranks})$.
- Calculate the p-value for the Wilcoxon signed-rank statistic using a table or a computer program.

If the p-value is less than the significance level, then the null hypothesis can be rejected and it can be concluded that there is a significant difference in the distribution of the variable before and after the intervention, or between the two paired groups.

> We'll explore the difference between the proportion of county-level votes for the Democratic candidate in 2012 and 2016 to identify if the difference is significant. We will use a reduced version of the original dataset which is sampled randomly to include only 10 rows so that the conditions for paired t-test are not met (no of pairs >= 30). First we will use a normal paired t-test and then We'll use the Wilcoxon signed-rank test to check if the difference is significant.

In [9]:
alpha = 0.05

In [11]:
import pingouin

In [13]:
reduced_sample_dem_data = sample_dem_data.sample(n=10, random_state=327)

- Normal paired t-test

In [15]:
# Conduct a paired t-test on dem_percent_12 and dem_percent_16
paired_test_results = pingouin.ttest(
    x=reduced_sample_dem_data["dem_percent_12"],
    y=reduced_sample_dem_data["dem_percent_16"],
    alternative="two-sided",
    paired=True,
)

# Print paired t-test results
paired_test_results

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,5.640869,9,two-sided,0.000317,"[4.6, 10.77]",0.583642,107.095,0.378369


- Wilcoxon signed-rank test

We can use the `scipy.stats.wilcoxon()` method to perform the Wilcoxon signed-rank test. We can also use the `pingouin.wilcoxon()` function from the Pingouin module to perform the Wilcoxon signed-rank test. Both of these functions takes the two paired groups as input and returns the test statistic and the p-value.

In [17]:
# Conduct a Wilcoxon test on dem_percent_12 and dem_percent_16
wilcoxon_test_results = pingouin.wilcoxon(
    x=reduced_sample_dem_data["dem_percent_12"],
    y=reduced_sample_dem_data["dem_percent_16"],
    alternative="two-sided",
)

# Print Wilcoxon test results
wilcoxon_test_results

,W-val,alternative,p-val,RBC,CLES
Wilcoxon,0.0,two-sided,0.001953,1.0,0.73


We can see from these results that the p-value in wilcoxon test is greater than the result found in the paired t-test.